# util

## deco.py

In [ ]:
from functools import wraps


def constrain(*allowed_args):
    """
    함수의 입력 인자가 1개이고 해당 인자의 가능한 값을 미리 정의

    사용 예시)
        @constrain("ICE", "HEV")
        def developerDB(engineSpec:str) -> None:
            ...

    :param allowed_args:

    :return:
    """
    allowed_set = set(allowed_args)
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            value = args[1] if len(args) > 1 else kwargs.get('value')

            if value not in allowed_set:
                raise ValueError(f"입력한 값: {value}은 입력 가능한 인자: {allowed_args}가 아닙니다")

            return func(*args, **kwargs)
        return wrapper
    return decorator

## logger.py

In [ ]:
from io import StringIO
from time import localtime, mktime, gmtime
import logging, sys


class Logger(logging.Logger):
    _buffer = None
    _format = f"%(asctime)s %(message)s"

    @classmethod
    def kst(cls, *args):
        return localtime(mktime(gmtime()) + 9 * 3600)

    def __init__(self, name: str):
        super().__init__(name=name, level=logging.INFO)
        self.propagate = False

        formatter = logging.Formatter(
            fmt=self._format,
            datefmt="%Y-%m-%d %H:%M:%S"
        )
        formatter.converter = self.kst

        stream = logging.StreamHandler(stream=sys.stdout)
        stream.setLevel(logging.INFO)
        stream.setFormatter(formatter)
        self.addHandler(stream)

        self._buffer = StringIO()
        memory = logging.StreamHandler(stream=self._buffer)
        memory.setLevel(logging.INFO)
        memory.setFormatter(formatter)
        self.addHandler(memory)
        return

    def __call__(self, msg: str):
        self.info(msg=msg)
        return

    @property
    def formatter(self):
        return self._format

    @formatter.setter
    def formatter(self, formatter:str):
        self._format = formatter
        if formatter.startswith("%(asctime)s") or formatter.lower() == "default":
            formatter = logging.Formatter(fmt=formatter, datefmt="%Y-%m-%d %H:%M:%S")
            formatter.converter = self.kst
        for handler in self.handlers:
            handler.setFormatter(logging.Formatter(fmt=formatter))
        return

    @property
    def stream(self) -> str:
        return self._buffer.getvalue()

    def to_html(self):
        return self.stream \
               .replace("\n", "<br>") \
               .replace("  -", f'{"&nbsp;" * 4}-') \
               .replace("---", "<hr>")

    def clear(self):
        self._buffer.truncate(0)
        self._buffer.seek(0)
        return

## mailing.py

In [ ]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP


class Mail(MIMEMultipart):
    """
    e-mail 전송 서비스
    """

    def __init__(self):
        super().__init__()
        self['From'] = 'snob.labwons@gmail.com'
        self.content = ""
        return

    @property
    def Subject(self) -> str:
        return self['Subject']

    @Subject.setter
    def Subject(self, _subject: str):
        self['Subject'] = _subject

    @property
    def To(self) -> str:
        return self['To']

    @To.setter
    def To(self, _to: str):
        self['To'] = _to

    @classmethod
    def to_html(cls, text: str) -> str:
        return f"""
<!doctype html>
<html>    
    <style>
        .styled-table {{
            border-collapse: collapse;
            width: 100%;
            text-align: right;
        }}
        .styled-table th, .styled-table td {{
            border: 1px solid #ccc;
            padding: 8px;
            font-weight: 500;
        }}
    </style>
    <body>
        <p>{text}</p>
    </body>
</html>
"""

    def send(self, *args):
        self.attach(MIMEText(self.content, *args))
        with SMTP('smtp.gmail.com', 587) as server:
            server.ehlo()
            server.starttls()
            server.login(self['From'], "puiz yxql tnoe ivaa")
            server.send_message(self)
        return

# crypto

## bithumb

### ticker.py

In [ ]:
if not "constrain" in globals():
    from src.util.deco import constrain
if not "Logger" in globals():
    from src.util.logger import Logger
from pandas import DataFrame, Series
from typing import Union
import pandas as pd
import requests


class Ticker:
    url: str = "https://api.bithumb.com/v1"
    headers = {"accept": "application/json"}
    rename = {
        "candle_date_time_kst": "datetime",
        "opening_price": "open",
        "high_price": "high",
        "low_price": "low",
        "trade_price": "close",
        "candle_acc_trade_price": "amount",
        "candle_acc_trade_volume": "volume",
        "trade_volume": 'volume',
        "ask_bid": "quote",
        "datetime": "datetime"
    }

    def __init__(self, ticker: str):
        self.ticker = ticker
        self._snap = Series()
        return

    def __repr__(self):
        return repr(self.snapShot)

    def __getitem__(self, item):
        if self._snap.empty:
            self._snap = self.snapShot
        return self._snap[item]

    @classmethod
    def _fetch_(cls, url: str, **kwargs) -> Union[DataFrame, Series]:
        """
        url: https://api.bithumb.com 의 api 데이터 취득
        base url의 하위 주소를 입력하여 응답을 pandas Series 또는 DataFrame으로 변환

        Args:
            url     (str)   : 주소, base url이 제외된 하위 주소
            kwargs  (dict)  : Series 또는 DataFrame 변환 시 전달할 Keywoard Arguments

        Returns:
            Union[Series, DataFrame] :
        """
        resp = requests \
            .get(f"{cls.url}{url}", headers=cls.headers) \
            .json()
        return Series(resp[0], **kwargs) if len(resp) == 1 else DataFrame(resp, **kwargs)

    @property
    def snapShot(self) -> Series:
        """
        market                               KRW-BTC
        trade_date                          20251128
        trade_time                            021901
        trade_date_kst                      20251128
        trade_time_kst                        111901
        trade_timestamp                1764328741157
        opening_price                      135796000
        high_price                         137125000
        low_price                          135372000
        trade_price                        135442000
        prev_closing_price                 135796000
        change                                  FALL
        change_price                          354000
        change_rate                           0.0026
        signed_change_price                  -354000
        signed_change_rate                   -0.0026
        trade_volume                        0.000073
        acc_trade_price           29804029073.873871
        acc_trade_price_24h      111366853330.307678
        acc_trade_volume                  218.675501
        acc_trade_volume_24h              815.641639
        highest_52_week_price              179734000
        highest_52_week_date              2025-10-10
        lowest_52_week_price               110000000
        lowest_52_week_date               2024-12-04
        timestamp                      1764328741157
        Name: KRW-BTC, dtype: object
        """
        self._snap = self._fetch_(f"/ticker?markets={self.ticker}", name=self.ticker)
        return self._snap

    # @constrain('1minutes', '3minutes', '5minutes', '10minutes',
    #            '15minutes', '30minutes', '60minutes', '240minutes',
    #            '1days', '1weeks', '1months')
    def ohlcv(self, interval: str, to: str = '') -> DataFrame:
        """
                                 open	     high	      low	    close	      amount	     volume
        datetime
        2025-12-08T14:30:00	135976000	136192000	135976000	136032000	3.107575e+08	 2283670.13
        2025-12-08T14:00:00	135976000	136167000	135544000	135975000	1.790029e+09	13176614.54
        2025-12-08T13:30:00	135948000	136185000	135811000	135977000	1.024618e+09	 7535912.03
        2025-12-08T13:00:00	135782000	136038000	135700000	135948000	7.361658e+08	 5419830.47
        2025-12-08T12:30:00	136113000	136113000	135702000	135761000	1.072309e+09	 7893673.71
        ...	...	...	...	...	...	...
        2025-12-04T13:00:00	139211000	139232000	138978000	139048000	1.259391e+09	 9056382.13
        2025-12-04T12:30:00	139232000	139250000	138999000	139199000	1.363260e+09	 9800596.39
        2025-12-04T12:00:00	139559000	139650000	139160000	139244000	1.809427e+09	12977284.73
        2025-12-04T11:30:00	139374000	139618000	139148000	139559000	2.374796e+09	17033654.61
        2025-12-04T11:00:00	138590000	139590000	138500000	139374000	4.618125e+09	33227210.66
        200 rows × 6 columns

        :param interval:
        :param count:
        :return:
        """
        if interval.endswith('minutes'):
            query = f'/candles/minutes/{interval.replace("minutes", "")}?market={self.ticker}&count=200'
        else:
            query = f'/candles/{interval[1:]}?market={self.ticker}&count=200'

        if to:
            query += f'&to={to}'

        data = self._fetch_(query)
        if isinstance(data, Series):
            raise KeyError
        cols = {k: v for k, v in self.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data.set_index(keys='datetime')
        data['volume'] = data['volume'] * 1e+6
        return data.sort_index(ascending=True)

    def execution(self, count: int = 100) -> DataFrame:
        """
        체결 내역

                                close	  volume  quote
        datetime
        2025-12-08 14:42:39	136108000	0.005988	BID
        2025-12-08 14:42:39	136100000	0.003500	BID
        2025-12-08 14:42:39	136099000	0.000087	BID
        2025-12-08 14:42:39	136082000	0.003211	BID
        2025-12-08 14:42:21	136082000	0.000050	BID
        ...	...	...	...
        2025-12-08 14:39:29	136101000	0.000073	ASK
        2025-12-08 14:39:29	136107000	0.001832	BID
        2025-12-08 14:39:18	136107000	0.003049	BID
        2025-12-08 14:39:10	136107000	0.000004	ASK
        2025-12-08 14:39:07	136108000	0.000062	ASK
        100 rows × 3 columns

        :param count:
        :return:
        """
        data = self._fetch_(f'/trades/ticks?market={self.ticker}&count={min(count, 500)}')
        data['datetime'] = pd.to_datetime(data['trade_date_utc'].astype(str) + ' ' + data['trade_time_utc'].astype(str))
        data['datetime'] = data['datetime'] + pd.Timedelta(hours=9)
        cols = {k: v for k, v in self.rename.items() if k in data.columns}

        data = data.rename(columns=cols)[cols.values()]
        data = data.set_index(keys='datetime')
        return data

    def order(self) -> DataFrame:
        """
        주문 내역

                            ask_price	bid_price  ask_size  bid_size
        datetime
        2025-12-08 14:43:42	136107000	136082000	 0.0012    0.0007
        2025-12-08 14:43:42	136108000	136059000	 0.0356	   0.0735
        2025-12-08 14:43:42	136109000	136026000	 0.1031	   0.0043
        2025-12-08 14:43:42	136112000	136025000	 0.0328	   0.0346
        2025-12-08 14:43:42	136117000	136014000	 0.0036	   0.0029
        ... ... ... ... ...
        2025-12-08 14:43:42	136182000	135977000	 0.0038	   0.0588
        2025-12-08 14:43:42	136185000	135976000	 0.0093	   0.0907
        2025-12-08 14:43:42	136186000	135975000	 0.0322	   0.1778
        2025-12-08 14:43:42	136188000	135974000	 0.0007	   0.0000
        2025-12-08 14:43:42	136192000	135973000	 0.0010	   0.0722
        :return:
        """
        base = self._fetch_(f'/orderbook?markets={self.ticker}')
        data = DataFrame(base['orderbook_units'])
        data['datetime'] = pd.to_datetime(base['timestamp'], unit='ms') + pd.Timedelta(hours=9)
        data['datetime'] = data['datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
        return data.set_index(keys='datetime')

    def to_logger(self, logger: Logger):
        ticker = self.ticker.replace('KRW-', '')
        logger(f'TICKER: <a href="https://m.bithumb.com/react/trade/chart/{ticker}-KRW">{ticker}</a>')
        logger(f'  - 현재가: {self["trade_price"]}원')
        logger(f'  - 등락률: {100 * self["signed_change_rate"]:.2f}%')
        logger(f'  - 거래대금: {self["acc_trade_price_24h"] / 1e+8:.2f}억원')
        logger(f'---')
        return


### market.py

In [ ]:
if not "Ticker" in globals():
    from src.crypto.bithumb.ticker import Ticker
from pandas import DataFrame, Series
from typing import Union
import pandas as pd
import requests


class Market:
    url: str = "https://api.bithumb.com/v1"
    headers = {"accept": "application/json"}

    rename = {
        'market': 'ticker',
        'english_name': 'name',
        # 'market_warning': 'warning',
        # 'korean_name': 'kor',
        'warning_type': 'warning',
        'end_date': 'warning_end'
    }

    def __init__(self):
        self._baseline = DataFrame()
        self._failures = []
        return

    def __iter__(self):
        for ticker in self.tickers.index:
            yield ticker

    def _repr_html_(self):
        return getattr(self.tickers, '_repr_html_')()

    @classmethod
    def _fetch_(cls, url: str, **kwargs) -> Union[DataFrame, Series]:
        """
        url: https://api.bithumb.com 의 api 데이터 취득
        base url의 하위 주소를 입력하여 응답을 pandas Series 또는 DataFrame으로 변환

        Args:
            url     (str)   : 주소, base url이 제외된 하위 주소
            kwargs  (dict)  : Series 또는 DataFrame 변환 시 전달할 Keywoard Arguments

        Returns:
            Union[Series, DataFrame] :
        """
        resp = requests \
            .get(f"{cls.url}{url}", headers=cls.headers) \
            .json()
        return Series(resp[0], **kwargs) if len(resp) == 1 else DataFrame(resp, **kwargs)

    @classmethod
    def _fetch_tickers(cls) -> DataFrame:
        data = cls._fetch_('/market/all?isDetails=true')
        cols = {k: v for k, v in cls.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data[data['ticker'].str.startswith('KRW')]
        # data["warning"] = data["warning"].replace("NONE", None)
        return data.set_index(keys='ticker')

    @classmethod
    def _fetch_warnings(cls) -> DataFrame:
        data = cls._fetch_('/market/virtual_asset_warning')
        cols = {k: v for k, v in cls.rename.items() if k in data.columns}
        data = data.rename(columns=cols)[cols.values()]
        data = data[data['ticker'].str.startswith('KRW')]
        return data.set_index(keys='ticker')

    @property
    def baseline(self) -> DataFrame:
        if self._baseline.empty:
            self._baseline = self.update_baseline(interval='60minutes')
        return self._baseline

    @baseline.setter
    def baseline(self, baseline:DataFrame):
        self._baseline = baseline

    @property
    def failures(self) -> list:
        return self._failures

    @property
    def tickers(self) -> DataFrame:
        """
                            name	                          warning	        warning_end
        ticker
        KRW-BTC	         Bitcoin	                              NaN	                NaN
        KRW-ETH	        Ethereum	                              NaN	                NaN
        KRW-ETC	Ethereum Classic	                              NaN	                NaN
        KRW-XRP	             XRP	                              NaN	                NaN
        KRW-BCH	    Bitcoin Cash	                              NaN	                NaN
        ...	                 ...	                              ...	                ...
        KRW-MMT	        Momentum	                              NaN	                NaN
        KRW-MET	         Meteora	                              NaN	                NaN
        KRW-KITE	        Kite	                              NaN	                NaN
        KRW-TRUST	   Intuition	DEPOSIT_AMOUNT_SUDDEN_FLUCTUATION	2025-12-10 07:04:59
        KRW-PIEVERSE	Pieverse	                              NaN	                NaN
        450 rows × 3 columns
        """
        return self._fetch_tickers().join(self._fetch_warnings())

    def update_baseline(self, interval: str, to:str='') -> DataFrame:
        objs = {}
        for ticker in self.tickers.index:
            try:
                objs[ticker] = Ticker(ticker).ohlcv(interval, to=to)
            except KeyError:
                self._failures.append(ticker)
                continue
        self._baseline = pd.concat(objs, axis=1).sort_index(ascending=True).tail(200)
        return self._baseline

    def reset_failures(self):
        self._failures = []
        return

# analysis

## indicator.py

In [ ]:
from pandas import DataFrame
import pandas as pd


class Indicator:

    def __init__(self, baseline: DataFrame):
        self.data = baseline.copy()
        return

    def __call__(self, *tickers) -> DataFrame:
        return self.data.loc[:, pd.IndexSlice[list(tickers), :]]

    def __iter__(self):
        for col in self.data.columns.get_level_values(0).unique():
            yield col

    def __contains__(self, col: str):
        return col in self.data['KRW-BTC'].columns

    def __getitem__(self, col: str):
        return self.data.xs(col, axis=1, level=1)

    def __setitem__(self, col: str, series):
        series.columns = pd.MultiIndex.from_product([series.columns, [col]])
        self.data = pd.concat([self.data, series], axis=1).sort_index(axis=1)
        return

    def __delitem__(self, col: str):
        if not col in self:
            return
        mask = self.data.columns.get_level_values(1) == col
        self.data = self.data.drop(columns=self.data.columns[mask])
        return

    def _repr_html_(self):
        return self.data._repr_html_()

    def _set_columns(self, **kwargs):
        for column, series in kwargs.items():
            self[column] = series
        return

    def _del_columns(self, *cols):
        for col in cols:
            del self[col]
        return

    def install(self):
        self.add_tp()
        self.add_bb()
        self.add_macd()
        return

    def add_tp(self):
        self['tp'] = round((self['high'] + self['low'] + self['close']) / 3, 4)
        return

    def add_bb(self, basis: str = 'tp', window: int = 20, std: int = 2):
        basis = basis if basis in self else 'close'
        dev = self[basis].rolling(window).std()
        mid = self[basis].rolling(window).mean()
        up = mid + std * dev
        dn = mid - std * dev
        self._set_columns(
            mid=mid,
            bb_upper=up,
            bb_lower=dn,
            tr_upper=mid + (std / 2) * dev,
            tr_lower=mid - (std / 2) * dev,
            bb_width=((up - dn) / mid) * 100
        )
        return

    def add_macd(
        self,
        basis:str='tp',
        window_slow: int = 26,
        window_fast: int = 12,
        window_sign: int = 9,
    ):
        ema = lambda series, window: series.ewm(span=window).mean()
        fast = ema(self[basis], window_fast)
        slow = ema(self[basis], window_slow)
        macd = fast - slow
        sig = ema(macd, window_sign)
        self._set_columns(
            macd=macd,
            macd_signal=sig,
            macd_diff=macd - sig
        )
        return

# indicator = Indicator(coins.baseline)
# indicator.install()
# indicator.add_tp()
# indicator.add_bb()
# indicator

# indicator['amount']

## strategy.py

In [ ]:
if not "Indicator" in globals():
    from src.analysis.indicator import Indicator
from pandas import Series, DataFrame
import pandas as pd


class Strategy(Indicator):

    @staticmethod
    def describe(signal:DataFrame):
        signaled = []
        for ticker in signal.columns:
            unit = signal[ticker].dropna()
            if unit.empty:
                continue
            signaled.append(unit)
        return pd.concat(signaled, axis=1).sort_index(ascending=True)

    def squeeze_expand(
        self,
        window_width:int=100,
        width_threshold:float=0.1,
        window_volume:int=20,
        describe:bool=True,
    ) -> DataFrame:
        # 변동성 확대 국면 파악
        self['bb_squeeze'] = self['bb_width'] \
                             .rolling(window=window_width) \
                             .apply(lambda x:Series(x).rank(pct=True).iloc[-1], raw=False) < width_threshold
        self['bb_squeeze_release'] = self['bb_squeeze'].shift(1) & (~self['bb_squeeze'])
        self['bb_breakout'] = self['close'] > self['bb_upper']
        self['volume_spike'] = self['volume'] > self['volume'].rolling(window_volume).mean()
        self['sig_squeeze_expand'] = (
            self['bb_squeeze_release'] &
            self['bb_breakout'] &
            self['volume_spike']
        )
        del self['bb_squeeze']
        del self['bb_squeeze_release']
        del self['bb_breakout']
        del self['volume_spike']
        sig = self['sig_squeeze_expand'].astype(int).replace(0, None)
        if not describe:
            return sig
        return self.describe(signal=sig)

    def drawdown_recover(
        self,
        basis:str='tp',
        window:int=36,
        drawdown_threshold:float=-0.1,
        drawdown_recover_threshold:float=0.3,
        drawdown_rapid:int=3,
        describe:bool=True,
    ) -> DataFrame:
        self['dd_max'] = self[basis].rolling(window=window).max()
        self['dd_min'] = self[basis].rolling(window=window).min()
        self['dd_rapid'] = (
                (self['close'].pct_change(1, fill_method=None) <= (drawdown_threshold / 3)) |
                (self['close'].pct_change(drawdown_rapid, fill_method=None) <= (drawdown_threshold / 2))
        ).astype(int)
        self['dd_occur'] = (
            # 급락 감지
            (
                # 종가 기준 1봉 변화량이 허용 낙폭을 넘어갈 때
                (self['close'].pct_change(1, fill_method=None) <= (drawdown_threshold / 3)) |
                # 종가 기준 n봉
            )
            
            
            
           (self['dd_rapid'].rolling(window).sum() >= 1) &
           ((self['dd_min'] / self['dd_max'] - 1) <= drawdown_threshold) &
           ((self['close'] > self['tr_upper']).astype(int).rolling(window).sum() >= 1)
        )
        self['dd_recover'] = (self[basis] - self['dd_min']) / (self['dd_max'] - self['dd_min'])

        self['is_macd_pos'] = self['macd_diff'] >= 0
        self['macd_cross'] = self['is_macd_pos'] & (~self['is_macd_pos'].shift(1).astype(bool))

        self['sig_drawdown_recover'] = (
            self['dd_occur'] &
            (self['dd_recover'] <= drawdown_recover_threshold) &
            self['macd_cross']
        )

        del self['dd_max']
        del self['dd_min']
        del self['dd_recover']
        del self['dd_rapid']
        del self['dd_occur']
        del self['is_macd_pos']
        del self['macd_cross']
        sig = self['sig_drawdown_recover'].astype(int).replace(0, None)
        if not describe:
            return sig
        return self.describe(signal=sig)




# bot

## book

### tradingbook.py

In [ ]:
if not "Ticker" in globals():
    from src.crypto.bithumb.ticker import Ticker
from datetime import datetime, timedelta
from pandas import DataFrame
from numpy import nan
import pandas as pd
import os, requests, io, certifi


SCHEMA = {
    'ticker': { 'index': True, 'dtype': str , 'default': ''},
    'status': {'index': False, 'dtype': str, 'default': ''},
    'current_price': {'index': False, 'dtype': float, 'default': nan},
    'current_amount': {'index': False, 'dtype': float, 'default': nan},
    'current_volume': {'index': False, 'dtype': float, 'default': nan},
    'buy_price': {'index': False, 'dtype': float, 'default': nan},
    'buy_time': {'index': False, 'dtype': str, 'default': ''},
    'sell_price': {'index': False, 'dtype': float, 'default': nan},
    'sell_time': {'index': False, 'dtype': str, 'default': ''},
    'signal': {'index': False, 'dtype': str, 'default': ''},
    'signaled_time': {'index': False, 'dtype': str, 'default': ''},
    'signaled_price': {'index': False, 'dtype': float, 'default': nan},
    'signal_reported_price': {'index': False, 'dtype': float, 'default': nan},
    'signaled_amount': {'index': False, 'dtype': float, 'default': nan},
    'signaled_volume': {'index': False, 'dtype': float, 'default': nan},
    'yield_confirmed': {'index': False, 'dtype': float, 'default': nan},
    'yield_ongoing': {'index': False, 'dtype': float, 'default': nan},
    'yield_1h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_4h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_12h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_24h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_36h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_48h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_60h_from_detected': {'index': False, 'dtype': float, 'default': nan},
    'yield_72h_from_detected': {'index': False, 'dtype': float, 'default': nan},
}
STATUS = [
    "WATCH",
    "HOLD",
    "BID",
    "ASK",
    "SELL"
]
class TradingBook:

    _filename:str = 'book.json'
    try:
        _basepath:str = os.path.dirname(__file__)
    except NameError:
        _basepath:str = os.getcwd()
    _filepath:str = os.path.join(_basepath,_filename)

    def __init__(self, readonly:bool=False):
        if readonly:
            url = (
                "https://raw.githubusercontent.com"
                "/labwons"
                "/labwons-analytic"
                "/refs"
                "/heads"
                "/main"
                "/src"
                "/bot"
                "/book"
                "/book.json"
            )
            try:
                resp = requests.get(url, verify=certifi.where())
            except requests.exceptions.SSLError:
                resp = requests.get(url, verify=False)
            self.book = pd.read_json(io.StringIO(resp.text), orient='index')
        else:
            if not os.path.isfile(self._filepath):
                self.book = DataFrame(columns=list(SCHEMA.keys()))
            else:
                self.book = pd.read_json(self._filepath, orient="index")
                if self.book.empty:
                    self.book = DataFrame(columns=list(SCHEMA.keys()))
        if 'index' in self.book.columns:
            self.book = self.book.rename(columns={'index':'ticker'}).set_index(keys='ticker')
        return

    def __repr__(self):
        return repr(self.book)

    def __str__(self):
        return str(self.book)

    def __getattr__(self, item):
        return getattr(self.book, item)

    def __getitem__(self, item):
        return self.book[item]

    def __setitem__(self, key, value):
        return self.book.__setitem__(key, value)

    def _repr_html_(self):
        return getattr(self.book, '_repr_html_')()

    def append(self, ticker:str, **kwargs):
        new = DataFrame(
            index=[ticker],
            data=[{
                key: kwargs.get(key, schema['default']) for key, schema in SCHEMA.items()
            }]
        )
        self.book = pd.concat([self.book, new.drop(columns=['ticker'])], axis=0)
        return

    def update(self):
        self.book['signaled_time'] = self.book['signaled_time'].astype(str).str.replace("T", " ")
        for ticker in self.index:
            coin = Ticker(ticker=ticker)
            data = coin.ohlcv(interval='60minutes')

            s_price = self.loc[ticker, 'signaled_price']
            try:
                s_time = datetime.strptime(str(self.loc[ticker, 'signaled_time']), '%Y-%m-%d %H:%M:%S')
            except ValueError:
                continue
            for h in [1, 4, 12, 24, 36, 48, 60, 72]:
                e_time = s_time + timedelta(hours=h)
                if e_time.strftime('%Y-%m-%dT%H:%M:%S') not in data.index:
                    continue
                price_at_time = data.loc[e_time.strftime('%Y-%m-%dT%H:%M:%S'), 'close']
                self.loc[ticker, f'yield_{h}h_from_detected'] = 100 * (price_at_time - s_price) / s_price

            self.loc[ticker, 'current_price'] = curr = coin['trade_price']
            self.loc[ticker, 'current_amount'] = data.iloc[-1]['amount']
            self.loc[ticker, 'current_volume'] = data.iloc[-1]['volume']
            self.loc[ticker, 'yield_ongoing'] = 100 * (curr - s_price) / s_price
        return

    def save(self):
        # keys = list(SCHEMA.keys())
        # keys.remove('ticker')
        self.reset_index().to_json(self._filepath, orient="index")
        return



## master.py

In [ ]:
if not "Strategy" in globals():
    from src.analysis.strategy import Strategy
if not "Market" in globals():
    from src.crypto.bithumb.market import Market
if not "Ticker" in globals():
    from src.crypto.bithumb.ticker import Ticker
if not "Logger" in globals():
    from src.util.logger import Logger
if not "Mail" in globals():
    from src.util.mailing import Mail
if not "TradingBook" in globals():
    from src.bot.book.tradingbook import TradingBook
from datetime import datetime
from zoneinfo import ZoneInfo
from time import perf_counter
import os


utc = datetime.now(ZoneInfo('UTC'))
kst = datetime.now(ZoneInfo('Asia/Seoul'))

logger = Logger('BOT@v1')
logger.formatter = "%(message)s"
logger(f"RUNS ON: {os.getenv('EVENT_NAME', 'LOCAL').upper()}")

# BASELINE UPDATE
tic = perf_counter()
market = Market()
market.update_baseline(interval='60minutes')
market_time = datetime.strptime(market.baseline.index[-1], "%Y-%m-%dT%H:%M:%S")
if (market_time.hour > kst.hour) and (kst.minute < 45):
    market.baseline = market.baseline.iloc[:-1]
    market_time = datetime.strptime(market.baseline.index[-1], "%Y-%m-%dT%H:%M:%S")
for failed in market.failures:
    logger(f"⚠️  FAILED TICKER: {failed}")
market.reset_failures()
elapsed = perf_counter() - tic
logger(f"UPDATE BASELINE ... {int(elapsed // 60)}m {int(elapsed % 60)}s")
logger(f'KST: {kst.strftime("%Y/%m/%d %H:%M")}')
logger(f'MKT: {market_time.strftime("%Y/%m/%d %H:%M")}')

# INSTALL STRATEGY
strategy = Strategy(market.baseline)
strategy.install()


# REPORT SIGNALS
send = False
book = TradingBook()
for name, signal in [
    ("Squeeze & Expand",
     strategy.squeeze_expand(
        window_width=100,
        width_threshold=0.2,
        window_volume=20,
    )),
    ("Drawdown Recover",
     strategy.drawdown_recover(
        basis='tp',
        window=36,
        drawdown_threshold=-0.1,
        drawdown_recover_threshold=0.3,
        drawdown_rapid=3,
    )),
]:
    detect = signal.iloc[-1].dropna()
    if detect.empty:
        continue

    logger(f'<h1>{name}</h1>')
    logger(f'---')
    for n, ticker in enumerate(detect.index, start=1):
        coin = Ticker(ticker=ticker)
        coin.to_logger(logger)

        baseline = market.baseline[ticker].loc[detect.name]
        signaled_price = baseline['close']
        signal_reported_price = coin['trade_price']
        if abs(signal_reported_price / signaled_price - 1) >= 0.02:
            continue

        book.append(
            ticker=ticker,
            status='WATCH',
            signal=name,
            signaled_time=detect.name,
            signaled_price=signaled_price,
            signal_reported_price=signal_reported_price,
            signaled_amount=baseline['amount'],
            signaled_volume=baseline['volume'],
        )
    send = True

book.update()
book.save()

# SEND E-MAIL
if send:
    mail = Mail()
    mail.Subject = f'TRADER@v1 ON {kst.strftime("%Y/%m/%d %H:%M")}'
    mail.content = mail.to_html(logger.to_html())
    mail.To = ",".join([
        "jhlee_0319@naver.com",
        # "ghost3009@naver.com"
    ])
    mail.send("html", "utf-8")

else:
    logger('NO SIGNALS DETECTED ... SYSTEM ABORT')

